In [1]:
using CUDA, CUDA.CUSOLVER
using TypedTables
using DataFrames
using Statistics
using CSV
using LinearAlgebra

In [2]:
# CUDA

# function operation_matrices_cuda(A)
#     gpu_matrix = CUDA.cu(A)  # Transfer matrix to GPU
#     inverted_gpu_matrix = inv(gpu_matrix)  # Invert matrix on GPU
#     return CUDA.collect(inverted_gpu_matrix)  # Transfer back to CPU
# end

# Function to compute eigenvalues on the GPU
function operation_matrices_cuda(A, N)
    gpu_matrix = CUDA.cu(A)  # Transfer matrix to GPU
    # Compute eigenvalues (CUDA.jl does not directly support eigenvalue computations,
    # so this is a placeholder for analogous operations or library calls)
    # eigenvalues = CUDA.linalg.eigvals(gpu_matrix)  # Placeholder function call
    eigenvalues = CUDA.zeros(eltype(A), N)
    # CUDA.CUSOLVER.syevd!('N', 'U', gpu_matrix, eigenvalues)
    return CUDA.collect(eigenvalues)  # Transfer eigenvalues back to CPU
end

function simulate_gpu(A_normal, N)
    # Convert your matrices to CuArray
    # A = CUDA.cu(A_normal)

    # Measure execution time
    cuda_start = CUDA.@elapsed result_gpu = operation_matrices_cuda(A_normal, N)
    return cuda_start
end

simulate_gpu (generic function with 1 method)

In [3]:
# CPU

# function operation_matrices_normal(A)
#     C = zeros(Float32, size(A))
#     C = inv(A)
#     return C
# end

# Function to compute eigenvalues on the CPU
function eigenvalues_cpu(A)
    return eigen(A).values
end

function simulate_cpu(A_normal)
    normal_start = @elapsed result_cpu = eigenvalues_cpu(A_normal)
    # println("Waktu eksekusi dengan CPU: $normal_start detik")
    return normal_start
end

simulate_cpu (generic function with 1 method)

In [4]:
function simulate(N::Int64, n_loop::Int64)
    println("Total size: $(N*N)")
    results_gpu = []
    results_cpu = []
    is_better_gpu = []
    for i in 1:n_loop
        A_normal = rand(Float32, N, N)
        # B_normal = rand(Float32, N, N)
        benchmark_gpu = simulate_gpu(A_normal, N)
        benchmark_cpu = simulate_cpu(A_normal)
        push!(results_gpu, benchmark_gpu)
        push!(results_cpu, benchmark_cpu)
        push!(is_better_gpu, benchmark_gpu < benchmark_cpu)
    end
    println("Mean CPU: $(mean(results_cpu))")
    println("Mean GPU: $(mean(results_gpu))")
    return DataFrame(CPU=results_cpu, GPU=results_gpu, Better_GPU=is_better_gpu)
end

simulate (generic function with 1 method)

In [4]:
# Function to compute eigenvalues on the CPU
function eigen_cpu(A)
    return eigvals(A)
end

# Placeholder function to simulate GPU eigenvalue computation
function eigen_gpu(A_gpu)
    # Convert GPU array back to a CPU array and cast to Float64 for computation
    A_cpu = Array{Float64}(A_gpu)
    return eigvals(A_cpu)
end

# Simulate operation on CPU
function simulate_cpu(A_normal)
    cpu_start = @elapsed result_cpu = eigen_cpu(A_normal)
    return cpu_start
end

# Simulate operation on GPU using placeholder
function simulate_gpu(A_normal)
    A_gpu = CUDA.cu(A_normal)
    gpu_start = CUDA.@elapsed result_gpu = eigen_gpu(A_gpu)
    return gpu_start
end

function simulate(N::Int64, n_loop::Int64)
    println("Matrix size: $(N)x$(N)")
    results_gpu = []
    results_cpu = []
    is_better_gpu = []
    for i in 1:n_loop
        A_normal = rand(Float64, N, N)  # Use Float64 for better precision in eigenvalue calculations
        benchmark_gpu = simulate_gpu(A_normal)
        benchmark_cpu = simulate_cpu(A_normal)
        push!(results_gpu, benchmark_gpu)
        push!(results_cpu, benchmark_cpu)
        push!(is_better_gpu, benchmark_gpu < benchmark_cpu)
    end
    println("Mean CPU time: $(mean(results_cpu))")
    println("Mean GPU time: $(mean(results_gpu))")
    return DataFrame(CPU=results_cpu, GPU=results_gpu, Better_GPU=is_better_gpu)
end

simulate (generic function with 1 method)

In [6]:
df = simulate(10, 10)
CSV.write("./1.csv", df)
df

Matrix size: 10x10
Mean CPU time: 2.67146e-5
Mean GPU time: 5.0716793e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,4.6941e-5,0.000118432,false
2,2.4946e-5,4.88e-5,false
3,2.4644e-5,4.4448e-5,false
4,2.486e-5,4.256e-5,false
5,2.4599e-5,4.1984e-5,false
6,2.3517e-5,4.272e-5,false
7,2.3401e-5,4.0928e-5,false
8,2.6571e-5,4.4128e-5,false
9,2.3646e-5,4.1344e-5,false


In [18]:
df = simulate(50, 10)
CSV.write("./2.csv", df)
df

Matrix size: 50x50
Mean CPU time: 0.0007598637
Mean GPU time: 0.0007910561


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000631315,0.000723072,false
2,0.000868031,0.000663168,true
3,0.000921584,0.000726528,true
4,0.00104189,0.00115709,false
5,0.000713696,0.00100576,false
6,0.000700826,0.000717536,false
7,0.00070687,0.000785344,false
8,0.000722648,0.000697376,true
9,0.000681085,0.000795008,false


In [23]:
df = simulate(100, 10)
CSV.write("./3.csv", df)
df

Matrix size: 100x100
Mean CPU time: 0.005174710900000001
Mean GPU time: 0.00484689


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.0109836,0.00871072,true
2,0.00562893,0.0042425,true
3,0.00437378,0.00416118,true
4,0.00534084,0.0042255,true
5,0.00445999,0.00433024,true
6,0.0044766,0.00509347,false
7,0.00368292,0.00413696,false
8,0.00407031,0.00474042,false
9,0.00456927,0.00410045,true


In [29]:
df = simulate(500, 10)
CSV.write("./4.csv", df)
df

Matrix size: 500x500
Mean CPU time: 0.22621564679999998
Mean GPU time: 0.21905796


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.318166,0.221644,true
2,0.217924,0.244743,false
3,0.232485,0.216691,true
4,0.211161,0.222412,false
5,0.217809,0.220374,false
6,0.209655,0.208358,true
7,0.212917,0.219806,false
8,0.218339,0.236126,false
9,0.176018,0.173416,true


In [31]:
df = simulate(1000, 10)
CSV.write("./5.csv", df)
df

Matrix size: 1000x1000
Mean CPU time: 1.0050245189
Mean GPU time: 1.0036176


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.862765,1.24286,false
2,0.908936,0.778609,true
3,0.792418,0.861367,false
4,1.00584,0.973661,true
5,1.01586,0.937266,true
6,0.862171,0.865705,false
7,1.17492,0.996656,true
8,0.908927,1.10976,false
9,1.08249,1.05117,true


In [ ]:
df = simulate(5000, 10)
CSV.write("./6.csv", df)
df

In [ ]:
df = simulate(10000, 10)
CSV.write("./7.csv", df)
df